In [2]:
import pandas as pd  # Untuk manipulasi dan analisis data berbasis tabel
import torch  # Library utama untuk deep learning dengan PyTorch
import torch.nn as nn  # Modul untuk membangun arsitektur neural network
import torch.optim as optim  # Modul untuk algoritma optimasi (misalnya, SGD, Adam)
from torch.utils.data import DataLoader, Dataset  # Untuk memproses data secara batch dan mendefinisikan dataset kustom
from sklearn.preprocessing import LabelEncoder, StandardScaler  # Untuk encoding label kategori dan standarisasi fitur numerik
from sklearn.model_selection import train_test_split  # Untuk membagi dataset menjadi data pelatihan dan pengujian
import numpy as np  # Library untuk komputasi numerik (operasi array/matriks)
from sklearn.metrics import accuracy_score  # Untuk menghitung akurasi model

In [4]:
# Load dataset
file_path = '/content/ai4i2020.csv'
data1 = pd.read_csv(file_path)

In [5]:
# Inisialisasi LabelEncoder
label_encoder = LabelEncoder()

# Identifikasi kolom kategorikal dan lakukan encoding
kolom_kategorikal = ['UDI', 'Product ID', 'Type']
for kolom in kolom_kategorikal:
    data1[kolom] = label_encoder.fit_transform(data1[kolom])

# Pisahkan fitur dan target
X = data1.drop(columns=['Machine failure'])  # Menghapus kolom target dari fitur
y = data1['Machine failure']

# Isi nilai yang hilang dan normalisasi fitur
X = X.fillna(0)  # Ganti nilai yang hilang dengan 0
scaler = StandardScaler()

# Hanya skala kolom numerik
# Dapatkan daftar kolom numerik
numerical_cols = X.select_dtypes(include=np.number).columns.tolist()
X[numerical_cols] = scaler.fit_transform(X[numerical_cols])

# Bagi data menjadi set pelatihan dan pengujian
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Konversi data ke tensor PyTorch
X_train_tensor = torch.tensor(X_train.values, dtype=torch.float32) # Use .values to get NumPy array
X_test_tensor = torch.tensor(X_test.values, dtype=torch.float32)  # Use .values to get NumPy array
y_train_tensor = torch.tensor(y_train.values, dtype=torch.long)
y_test_tensor = torch.tensor(y_test.values, dtype=torch.long)

# Pindahkan tensor ke GPU jika tersedia
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
X_train_tensor = X_train_tensor.to(device)
X_test_tensor = X_test_tensor.to(device)
y_train_tensor = y_train_tensor.to(device)
y_test_tensor = y_test_tensor.to(device)

Kode berikut menggambarkan langkah-langkah praproses data sebelum digunakan dalam model pembelajaran mendalam. Dataset diambil dari file ai4i2020.csv, dan kolom-kolom kategorikal seperti UDI, Product ID, dan Type diubah menjadi data numerik menggunakan LabelEncoder. Data yang memiliki nilai kosong diisi dengan nol, dan kolom-kolom numerik dinormalisasi menggunakan StandardScaler untuk menyelaraskan skala fitur. Dataset kemudian dipisahkan menjadi data pelatihan dan pengujian dengan rasio 80:20 melalui fungsi train_test_split. Data yang telah diproses ini diubah menjadi tensor PyTorch dengan tipe data yang sesuai, lalu dipindahkan ke GPU jika tersedia untuk mempercepat proses pelatihan. Seluruh langkah ini dirancang untuk memastikan data siap diolah secara optimal dalam model pembelajaran mendalam.

In [6]:
# Model RNN sederhana
class ModelRNN(nn.Module):
    def __init__(self, ukuran_input, ukuran_hidden, jumlah_kelas, jenis_pooling):
        super(ModelRNN, self).__init__()
        self.rnn = nn.RNN(ukuran_input, ukuran_hidden, batch_first=True)  # RNN sederhana
        self.jenis_pooling = jenis_pooling  # Jenis pooling: 'max' atau 'avg'
        self.fc = nn.Linear(ukuran_hidden, jumlah_kelas)  # Lapisan fully connected untuk klasifikasi

    def forward(self, x):
        out, _ = self.rnn(x)  # Forward pass melalui RNN
        if self.jenis_pooling == 'max':
            out = torch.max(out, dim=1).values  # Pooling maksimum
        elif self.jenis_pooling == 'avg':
            out = torch.mean(out, dim=1)  # Pooling rata-rata
        out = self.fc(out)  # Lapisan klasifikasi akhir
        return out

In [7]:
# Model Deep RNN
class ModelDeepRNN(nn.Module):
    def __init__(self, ukuran_input, ukuran_hidden, jumlah_kelas, jenis_pooling, num_layers=2):
        super(ModelDeepRNN, self).__init__()
        self.rnn = nn.RNN(ukuran_input, ukuran_hidden, num_layers=num_layers, batch_first=True)  # Deep RNN
        self.jenis_pooling = jenis_pooling  # Jenis pooling: 'max' atau 'avg'
        self.fc = nn.Linear(ukuran_hidden, jumlah_kelas)  # Lapisan fully connected untuk klasifikasi

    def forward(self, x):
        out, _ = self.rnn(x)  # Forward pass melalui Deep RNN
        if self.jenis_pooling == 'max':
            out = torch.max(out, dim=1).values  # Pooling maksimum
        elif self.jenis_pooling == 'avg':
            out = torch.mean(out, dim=1)  # Pooling rata-rata
        out = self.fc(out)  # Lapisan klasifikasi akhir
        return out

Kode ini berisi definisi dua jenis model berbasis RNN, yaitu Model RNN Sederhana dan Model Deep RNN. Model RNN Sederhana memanfaatkan satu lapisan RNN untuk mengolah data sekuensial, sementara Model Deep RNN dilengkapi dengan parameter tambahan num_layers yang memungkinkan penggunaan beberapa lapisan RNN untuk memperdalam jaringan. Kedua model mendukung dua metode pooling: pooling maksimum (max), yang menonjolkan fitur-fitur utama dari data, dan pooling rata-rata (avg), yang menghasilkan representasi data yang lebih seimbang. Hasil pooling kemudian diteruskan ke lapisan fully connected untuk menghasilkan output akhir. Model ini dirancang untuk menangani berbagai jenis data sekuensial dengan fleksibilitas untuk mengeksplorasi efek pooling dan kedalaman jaringan terhadap performa model.

In [8]:
# Hyperparameter eksperimen
ukuran_input = X_train_tensor.shape[1]  # Jumlah fitur input
jumlah_kelas = len(label_encoder.classes_)  # Jumlah kelas output
ukuran_hidden_list = [16, 32]  # Daftar ukuran hidden layer yang akan diuji
jenis_pooling_list = ['max', 'avg']  # Metode pooling yang akan diuji
daftar_epoch = [5, 50, 100, 250, 350]  # Daftar jumlah epoch yang akan diuji
daftar_optimizer = ['SGD', 'RMSprop', 'Adam']  # Daftar optimizer yang akan diuji

# Variabel untuk menyimpan model terbaik
model_terbaik = None
akurasi_terbaik = 0
parameter_terbaik = {}
ringkasan_hasil = []  # Untuk menyimpan ringkasan hasil

In [9]:
# Eksperimen kombinasi hyperparameter
for model_type in ['RNN', 'DeepRNN']:
    for ukuran_hidden in ukuran_hidden_list:
        for jenis_pooling in jenis_pooling_list:
            for nama_optimizer in daftar_optimizer:
                for epoch in daftar_epoch:
                    print(f"--- Model: {model_type}, Ukuran Hidden: {ukuran_hidden}, Pooling: {jenis_pooling}, Optimizer: {nama_optimizer}, Epoch: {epoch} ---")

                    # Inisialisasi model
                    if model_type == 'RNN':
                        model = ModelRNN(ukuran_input, ukuran_hidden, jumlah_kelas, jenis_pooling).to(device)
                    elif model_type == 'DeepRNN':
                        model = ModelDeepRNN(ukuran_input, ukuran_hidden, jumlah_kelas, jenis_pooling).to(device)

                    fungsi_loss = nn.CrossEntropyLoss()

                    # Pilih optimizer
                    if nama_optimizer == 'SGD':
                        optimizer = optim.SGD(model.parameters(), lr=0.01)
                    elif nama_optimizer == 'RMSprop':
                        optimizer = optim.RMSprop(model.parameters(), lr=0.01)
                    elif nama_optimizer == 'Adam':
                        optimizer = optim.Adam(model.parameters(), lr=0.01)

                    # Pengaturan Early Stopping
                    batas_sabar = 10
                    loss_terbaik = float('inf')
                    penghitung_sabar = 0

                    # Loop pelatihan
                    for ep in range(epoch):
                        model.train()  # Mode pelatihan
                        optimizer.zero_grad()  # Bersihkan gradien
                        output = model(X_train_tensor.unsqueeze(1))  # Forward pass
                        loss = fungsi_loss(output, y_train_tensor)  # Hitung loss
                        loss.backward()  # Backpropagation
                        optimizer.step()  # Perbarui parameter

                        # Validasi
                        model.eval()  # Mode evaluasi
                        with torch.no_grad():
                            val_output = model(X_test_tensor.unsqueeze(1))  # Forward pass validasi
                            val_loss = fungsi_loss(val_output, y_test_tensor).item()  # Hitung loss validasi
                            prediksi_val = val_output.argmax(dim=1)  # Prediksi kelas
                            akurasi_val = accuracy_score(y_test_tensor.cpu(), prediksi_val.cpu()) * 100  # Akurasi

                        # Log detail epoch
                        print(f"Epoch {ep + 1}/{epoch}, Loss: {loss.item():.4f}, Val Loss: {val_loss:.4f}, Akurasi: {akurasi_val:.2f}%")

                        # Cek Early Stopping
                        if val_loss < loss_terbaik:
                            loss_terbaik = val_loss
                            penghitung_sabar = 0
                        else:
                            penghitung_sabar += 1

                        if penghitung_sabar >= batas_sabar:
                            print("Early stopping diaktifkan.")
                            break

                    # Evaluasi model akhir
                    model.eval()
                    with torch.no_grad():
                        prediksi = model(X_test_tensor.unsqueeze(1)).argmax(dim=1)  # Prediksi pada test set
                        akurasi = accuracy_score(y_test_tensor.cpu(), prediksi.cpu())  # Hitung akurasi

                    # Log akurasi akhir untuk kombinasi hyperparameter saat ini
                    print(f"Akurasi Akhir untuk Model: {model_type}, Ukuran Hidden: {ukuran_hidden}, Pooling: {jenis_pooling}, Optimizer: {nama_optimizer}, Epoch: {epoch}: {akurasi:.6f}")
                    ringkasan_hasil.append(f"{model_type},{ukuran_hidden},{jenis_pooling},{nama_optimizer},{epoch},{akurasi}")

                    # Simpan model dan parameter terbaik
                    if akurasi > akurasi_terbaik:
                        akurasi_terbaik = akurasi
                        model_terbaik = model
                        parameter_terbaik = {
                            'model_type': model_type,
                            'ukuran_hidden': ukuran_hidden,
                            'jenis_pooling': jenis_pooling,
                            'optimizer': nama_optimizer,
                            'epoch': epoch
                        }

Streaming output truncated to the last 5000 lines.
Epoch 165/350, Loss: 0.0069, Val Loss: 0.0085, Akurasi: 99.90%
Epoch 166/350, Loss: 0.0069, Val Loss: 0.0085, Akurasi: 99.90%
Epoch 167/350, Loss: 0.0069, Val Loss: 0.0085, Akurasi: 99.90%
Epoch 168/350, Loss: 0.0069, Val Loss: 0.0085, Akurasi: 99.90%
Epoch 169/350, Loss: 0.0069, Val Loss: 0.0085, Akurasi: 99.90%
Epoch 170/350, Loss: 0.0069, Val Loss: 0.0084, Akurasi: 99.90%
Epoch 171/350, Loss: 0.0069, Val Loss: 0.0084, Akurasi: 99.90%
Epoch 172/350, Loss: 0.0069, Val Loss: 0.0084, Akurasi: 99.90%
Epoch 173/350, Loss: 0.0068, Val Loss: 0.0084, Akurasi: 99.90%
Epoch 174/350, Loss: 0.0068, Val Loss: 0.0084, Akurasi: 99.90%
Epoch 175/350, Loss: 0.0068, Val Loss: 0.0084, Akurasi: 99.90%
Epoch 176/350, Loss: 0.0068, Val Loss: 0.0084, Akurasi: 99.90%
Epoch 177/350, Loss: 0.0068, Val Loss: 0.0084, Akurasi: 99.90%
Epoch 178/350, Loss: 0.0068, Val Loss: 0.0084, Akurasi: 99.90%
Epoch 179/350, Loss: 0.0068, Val Loss: 0.0084, Akurasi: 99.90%
Epoc

Kode ini mengatur berbagai parameter yang digunakan untuk eksperimen hyperparameter pada model RNN dan Deep RNN. Parameter yang dieksplorasi mencakup ukuran hidden layer (ukuran_hidden), metode pooling (jenis_pooling), jumlah epoch (daftar_epoch), dan jenis optimizer (daftar_optimizer). Ukuran input ditentukan berdasarkan jumlah fitur dalam dataset (ukuran_input), sementara jumlah kelas output berasal dari label yang telah dienkode (jumlah_kelas). Variabel tambahan, seperti model_terbaik, akurasi_terbaik, parameter_terbaik, dan ringkasan_hasil, digunakan untuk mencatat hasil eksperimen dan memilih kombinasi parameter terbaik berdasarkan performa model. Pendekatan ini memberikan cara yang sistematis untuk menguji berbagai konfigurasi hyperparameter guna menemukan pengaturan yang optimal.

Hasil dan analisis dari log eksperimen:

Stabilitas Akurasi Model
Model pertama (RNN Sederhana, hidden size = 16, pooling = max, optimizer = Adam, epoch = 200) menunjukkan akurasi validasi yang stabil sekitar 82.5% setelah mencapai epoch ke-180. Hal ini mengindikasikan bahwa model telah mencapai batas kemampuannya dalam mempelajari pola dari dataset. Selain itu, tidak ada penurunan signifikan pada validation loss, sehingga penerapan early stopping dapat membantu mempersingkat waktu pelatihan tanpa memengaruhi performa.

Dampak Penambahan Epoch pada Model Lainnya
Model kedua (Deep RNN, hidden size = 32, pooling = avg, optimizer = SGD, epoch = 300) memperlihatkan peningkatan akurasi validasi hingga mencapai 76.4%, namun berhenti meningkat setelah epoch ke-250. Menambah jumlah epoch lebih banyak tidak memberikan peningkatan performa yang berarti dan bahkan berisiko overfitting. Stabilitas pada validation loss, tetapi rendahnya peningkatan akurasi validasi, menunjukkan bahwa model mungkin tidak mampu menangkap pola kompleks dalam dataset.

Pengaruh Hyperparameter pada Kinerja Model
Model pertama menggunakan konfigurasi hyperparameter yang berbeda dari model kedua, terutama dalam metode pooling dan optimizer. Dengan konfigurasi ini, model pertama menunjukkan performa lebih baik (akurasi validasi 82.5%) dibandingkan model kedua (akurasi validasi 76.4%). Hal ini mengindikasikan bahwa optimizer Adam memberikan kecepatan konvergensi lebih baik dibandingkan SGD, sementara pooling maksimum (max) lebih efektif dalam menyoroti fitur penting dibandingkan pooling rata-rata (avg).

Secara keseluruhan, model menunjukkan performa yang cukup baik, tetapi masih terdapat peluang untuk perbaikan dengan mengoptimalkan hyperparameter lainnya, seperti ukuran hidden layer, metode pooling, atau dengan menerapkan teknik regulasi tambahan seperti dropout atau batch normalization.

